In [1]:
import os
import glob
from osgeo import osr,gdal
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install ipython-autotime
%load_ext autotime

time: 1.01 ms (started: 2021-08-09 06:15:17 +00:00)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 23.7 s (started: 2021-08-09 06:15:17 +00:00)


In [3]:
photon_df = pd.read_csv("/content/drive/MyDrive/PS/PS Data/NSikkim/final.csv", index_col=0)

time: 7.29 s (started: 2021-08-09 06:15:40 +00:00)


/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
photon_df.head()

,latitude,longitude,photon height
index,,,
0,27.841415,88.454983,4309.2500
1,27.841344,88.454975,4311.4966
2,27.841325,88.454973,4312.4316
3,27.841313,88.454972,4310.0664
4,27.841287,88.454969,4311.5400


time: 19.8 ms (started: 2021-08-09 06:15:48 +00:00)


In [5]:
ds = gdal.Open("/content/drive/MyDrive/PS/PS Data/NSikkim/NSikkim_SRTMGL1Ellip.tif")
transform = ds.GetGeoTransform()
band = ds.GetRasterBand(1)
array = band.ReadAsArray()
# plt.imshow(array)
# array

time: 1.54 s (started: 2021-08-09 06:15:48 +00:00)


In [6]:
width = ds.RasterXSize
height = ds.RasterYSize
print(width)
print(height)

2321
1197
time: 3.36 ms (started: 2021-08-09 06:15:49 +00:00)


In [7]:
gt = ds.GetGeoTransform()

time: 1.36 ms (started: 2021-08-09 06:15:49 +00:00)


In [8]:
origin_x = gt[0]
origin_y = gt[3]
pixel_res_x = gt[1]
pixel_res_y = gt[5]

time: 2.39 ms (started: 2021-08-09 06:15:49 +00:00)


In [9]:
def latitude(n):
    return origin_y + (n/width * pixel_res_y)
def longitude(n):
    return origin_x + (n%width * pixel_res_x)

time: 2.45 ms (started: 2021-08-09 06:15:49 +00:00)


In [10]:
array = np.array(array)
array = array.flatten()
dem_df = pd.DataFrame(data = array,columns = ['dem height'])
dem_df['index1']=dem_df.index
dem_df['latitude'] = dem_df.index1.apply(latitude)
dem_df['longitude'] = dem_df.index1.apply(longitude)
dem_df = dem_df.drop(['index1'],axis=1)
dem_df = dem_df[['latitude','longitude','dem height']]

time: 2.12 s (started: 2021-08-09 06:15:49 +00:00)


In [11]:
dem_df

,latitude,longitude,dem height
0,27.874028,88.028194,5822.103516
1,27.874028,88.028472,5805.104492
2,27.874028,88.028750,5795.105469
3,27.874027,88.029028,5765.106445
4,27.874027,88.029306,5758.107422
...,...,...,...
2778232,27.541528,88.671528,1763.873535
2778233,27.541528,88.671806,1767.875000
2778234,27.541528,88.672083,1777.876465
2778235,27.541528,88.672361,1801.878052


time: 16.5 ms (started: 2021-08-09 06:15:52 +00:00)


In [12]:
def get_latitude_index(n):
  return int((n-origin_y)/pixel_res_y)

def get_longitude_index(n):
  return int((n-origin_x)/pixel_res_x)

def get_high_res_latitude_index(n):
  return int((n-origin_y)/(pixel_res_y/3))

def get_high_res_longitude_index(n):
  return int((n-origin_x)/(pixel_res_x/3))

time: 5.89 ms (started: 2021-08-09 06:15:52 +00:00)


In [13]:
photon_df['latitude_index']= photon_df.latitude.apply(get_latitude_index)
photon_df['longitude_index']= photon_df.longitude.apply(get_longitude_index)
photon_df['latitude_index2']= photon_df.latitude.apply(get_high_res_latitude_index)
photon_df['longitude_index2']= photon_df.longitude.apply(get_high_res_longitude_index)

time: 16.7 s (started: 2021-08-09 06:15:52 +00:00)


In [14]:
photon_df['container_box_id']=(photon_df['latitude_index'])*(width)+photon_df['longitude_index']

time: 61.9 ms (started: 2021-08-09 06:16:08 +00:00)


In [15]:
photon_df['container_box_id2']=(photon_df['latitude_index2'])*(width)+photon_df['longitude_index2']
photon_df

,latitude,longitude,photon height,latitude_index,longitude_index,latitude_index2,longitude_index2,container_box_id,container_box_id2
index,,,,,,,,,
0,27.841415,88.454983,4309.2500,117,1536,352,4609,273093,821601
1,27.841344,88.454975,4311.4966,117,1536,352,4609,273093,821601
2,27.841325,88.454973,4312.4316,117,1536,353,4609,273093,823922
3,27.841313,88.454972,4310.0664,117,1536,353,4609,273093,823922
4,27.841287,88.454969,4311.5400,117,1536,353,4609,273093,823922
...,...,...,...,...,...,...,...,...,...
170384,27.476968,88.414181,2974.6000,1429,1389,4288,4168,3318098,9956616
170386,27.476955,88.414180,2973.3796,1429,1389,4288,4168,3318098,9956616
170387,27.476949,88.414179,2970.8690,1429,1389,4288,4168,3318098,9956616


time: 279 ms (started: 2021-08-09 06:16:08 +00:00)


In [16]:
photon_df=photon_df.groupby(['container_box_id2']).mean()
photon_df

,latitude,longitude,photon height,latitude_index,longitude_index,latitude_index2,longitude_index2,container_box_id
container_box_id2,,,,,,,,
813184,27.841537,88.105476,5948.792200,116.0,278.0,350.0,834.0,269514.0
813362,27.841543,88.121953,5666.793253,116.0,337.0,350.0,1012.0,269573.0
813749,27.841543,88.157734,5713.768833,116.0,466.0,350.0,1399.0,269702.0
813796,27.841533,88.162106,5761.059586,116.0,482.0,350.0,1446.0,269718.0
814152,27.841550,88.195082,6397.295459,116.0,600.0,350.0,1802.0,269836.0
...,...,...,...,...,...,...,...,...
9961301,27.476857,88.633037,3145.156500,1429.0,2177.0,4289.0,6532.0,3318886.0
9961332,27.476860,88.635969,2953.649470,1429.0,2187.0,4289.0,6563.0,3318896.0
9961640,27.476866,88.664467,2560.722950,1429.0,2290.0,4289.0,6871.0,3318999.0


time: 856 ms (started: 2021-08-09 06:16:09 +00:00)


In [17]:
photon_df=photon_df.drop(['latitude_index'],axis=1)
photon_df=photon_df.drop(['longitude_index'],axis=1)
photon_df=photon_df.drop(['latitude_index2'],axis=1)
photon_df=photon_df.drop(['longitude_index2'],axis=1)
photon_df.reset_index(drop=True, inplace=True)

time: 25.8 ms (started: 2021-08-09 06:16:09 +00:00)


In [18]:
photon_df["container_box_id"] = photon_df["container_box_id"].astype(int)

time: 5.34 ms (started: 2021-08-09 06:16:10 +00:00)


In [19]:
photon_df

,latitude,longitude,photon height,container_box_id
0,27.841537,88.105476,5948.792200,269514
1,27.841543,88.121953,5666.793253,269573
2,27.841543,88.157734,5713.768833,269702
3,27.841533,88.162106,5761.059586,269718
4,27.841550,88.195082,6397.295459,269836
...,...,...,...,...
147853,27.476857,88.633037,3145.156500,3318886
147854,27.476860,88.635969,2953.649470,3318896
147855,27.476866,88.664467,2560.722950,3318999
147856,27.476889,88.665360,2485.663000,3319002


time: 24.5 ms (started: 2021-08-09 06:16:10 +00:00)


In [20]:
merged_df = photon_df.merge(dem_df, left_on='container_box_id', right_index = True)

time: 397 ms (started: 2021-08-09 06:16:10 +00:00)


In [21]:
merged_df

,latitude_x,longitude_x,photon height,container_box_id,latitude_y,longitude_y,dem height
0,27.841537,88.105476,5948.792200,269514,27.841772,88.105417,5977.253906
1,27.841543,88.121953,5666.793253,269573,27.841765,88.121806,5681.316895
2,27.841543,88.157734,5713.768833,269702,27.841750,88.157639,5724.455078
3,27.841533,88.162106,5761.059586,269718,27.841748,88.162083,5768.472168
4,27.841550,88.195082,6397.295459,269836,27.841734,88.194861,6399.598145
...,...,...,...,...,...,...,...
122428,27.541575,88.643069,1729.239986,2778129,27.541541,88.642917,1746.716919
122360,27.541769,88.657366,1986.384000,2778181,27.541534,88.657361,2029.795898
122392,27.541668,88.657377,2013.895333,2778181,27.541534,88.657361,2029.795898
122429,27.541582,88.657386,2000.582633,2778181,27.541534,88.657361,2029.795898


time: 30.6 ms (started: 2021-08-09 06:16:10 +00:00)


In [22]:
merged_df.drop(columns = ['container_box_id'], inplace = True)
X = merged_df.drop('photon height',axis=1)
y = merged_df['photon height']

time: 12.8 ms (started: 2021-08-09 06:16:10 +00:00)


In [23]:
from sklearn.model_selection import train_test_split

time: 723 ms (started: 2021-08-09 06:16:10 +00:00)


In [24]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=101)

time: 30.5 ms (started: 2021-08-09 06:16:11 +00:00)


In [25]:
import numpy as np
y_train = np.array(y_train)
y_test = np.array(y_test)

time: 3.41 ms (started: 2021-08-09 06:16:11 +00:00)


In [26]:
y_train = y_train.reshape(len(y_train), 1)
y_test = y_test.reshape(len(y_test), 1)

time: 2.64 ms (started: 2021-08-09 06:16:11 +00:00)


In [27]:
from sklearn.preprocessing import MinMaxScaler
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
X_train= scaler_x.fit_transform(X_train)
X_test = scaler_x.transform(X_test)
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

time: 24.5 ms (started: 2021-08-09 06:16:11 +00:00)


In [28]:
X_train.shape

(85665, 5)

time: 4.88 ms (started: 2021-08-09 06:16:11 +00:00)


In [29]:
X_test.shape

(36714, 5)

time: 4.68 ms (started: 2021-08-09 06:16:11 +00:00)


In [30]:
from sklearn.svm import SVR

time: 83.4 ms (started: 2021-08-09 06:16:11 +00:00)


In [31]:
model = SVR()

time: 1.61 ms (started: 2021-08-09 06:16:11 +00:00)


In [32]:
model.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

time: 4.14 s (started: 2021-08-09 06:16:11 +00:00)


In [33]:
finalarray=np.zeros([height*3*width*3], dtype = int)
df= pd.DataFrame(columns=['zero'],data=finalarray)
df

,zero
0,0
1,0
2,0
3,0
4,0
...,...
25004128,0
25004129,0
25004130,0
25004131,0


time: 45 ms (started: 2021-08-09 06:16:15 +00:00)


In [34]:
def latitude_final(n):
    return origin_y + (n/(width*3) * pixel_res_y/3.0)
def longitude_final(n):
    return origin_x + (n%(width*3) * pixel_res_x/3.0)

time: 2.65 ms (started: 2021-08-09 06:16:15 +00:00)


In [35]:
df['index1']=df.index
df['latitude']=df.index1.apply(latitude_final)
df['longitude']=df.index1.apply(longitude_final)

time: 21.7 s (started: 2021-08-09 06:16:15 +00:00)


In [36]:
df=df.drop(columns=['index1','zero'],axis=1)
df['latitude_index']= df.latitude.apply(get_latitude_index)
df['longitude_index']= df.longitude.apply(get_longitude_index)
df['container_box_id']=(df['latitude_index'])*(width)+df['longitude_index']
df=df.drop(['latitude_index'],axis=1)
df=df.drop(['longitude_index'],axis=1)
df["container_box_id"] = df["container_box_id"].astype(int)

time: 30.3 s (started: 2021-08-09 06:16:37 +00:00)


In [37]:
final_merged_df = df.merge(dem_df, left_on='container_box_id', right_index = True)
final_merged_df.drop(columns = ['container_box_id'], inplace = True)

time: 3.18 s (started: 2021-08-09 06:17:07 +00:00)


In [38]:
final_merged_df

,latitude_x,longitude_x,latitude_y,longitude_y,dem height
0,27.874028,88.028194,27.874028,88.028194,5822.103516
1,27.874028,88.028287,27.874028,88.028194,5822.103516
2,27.874028,88.028380,27.874028,88.028194,5822.103516
6963,27.873935,88.028194,27.874028,88.028194,5822.103516
6964,27.873935,88.028287,27.874028,88.028194,5822.103516
...,...,...,...,...,...
24997168,27.541620,88.672731,27.541528,88.672639,1862.879517
24997169,27.541620,88.672824,27.541528,88.672639,1862.879517
25004130,27.541528,88.672639,27.541528,88.672639,1862.879517
25004131,27.541528,88.672731,27.541528,88.672639,1862.879517


time: 25.1 ms (started: 2021-08-09 06:17:10 +00:00)


In [39]:
final_merged_df.sort_index()

,latitude_x,longitude_x,latitude_y,longitude_y,dem height
0,27.874028,88.028194,27.874028,88.028194,5822.103516
1,27.874028,88.028287,27.874028,88.028194,5822.103516
2,27.874028,88.028380,27.874028,88.028194,5822.103516
3,27.874028,88.028472,27.874028,88.028472,5805.104492
4,27.874028,88.028565,27.874028,88.028472,5805.104492
...,...,...,...,...,...
25004128,27.541528,88.672454,27.541528,88.672361,1801.878052
25004129,27.541528,88.672546,27.541528,88.672361,1801.878052
25004130,27.541528,88.672639,27.541528,88.672639,1862.879517
25004131,27.541528,88.672731,27.541528,88.672639,1862.879517


time: 2.82 s (started: 2021-08-09 06:17:10 +00:00)


In [40]:
X = final_merged_df
X = scaler_x.transform(X)

time: 1.22 s (started: 2021-08-09 06:17:13 +00:00)


In [41]:
predictions = model.predict(X)
final_merged_df['predictions']=predictions
final_merged_df

,latitude_x,longitude_x,latitude_y,longitude_y,dem height,predictions
0,27.874028,88.028194,27.874028,88.028194,5822.103516,0.607511
1,27.874028,88.028287,27.874028,88.028194,5822.103516,0.607527
2,27.874028,88.028380,27.874028,88.028194,5822.103516,0.607544
6963,27.873935,88.028194,27.874028,88.028194,5822.103516,0.607543
6964,27.873935,88.028287,27.874028,88.028194,5822.103516,0.607559
...,...,...,...,...,...,...
24997168,27.541620,88.672731,27.541528,88.672639,1862.879517,0.177591
24997169,27.541620,88.672824,27.541528,88.672639,1862.879517,0.177623
25004130,27.541528,88.672639,27.541528,88.672639,1862.879517,0.177594
25004131,27.541528,88.672731,27.541528,88.672639,1862.879517,0.177625


time: 2min 8s (started: 2021-08-09 06:17:14 +00:00)


In [42]:
final_merged_df['height']=scaler_y.inverse_transform(final_merged_df['predictions'].values.reshape(-1, 1))
final_merged_df

,latitude_x,longitude_x,latitude_y,longitude_y,dem height,predictions,height
0,27.874028,88.028194,27.874028,88.028194,5822.103516,0.607511,5626.679600
1,27.874028,88.028287,27.874028,88.028194,5822.103516,0.607527,5626.801687
2,27.874028,88.028380,27.874028,88.028194,5822.103516,0.607544,5626.923717
6963,27.873935,88.028194,27.874028,88.028194,5822.103516,0.607543,5626.916223
6964,27.873935,88.028287,27.874028,88.028194,5822.103516,0.607559,5627.038249
...,...,...,...,...,...,...,...
24997168,27.541620,88.672731,27.541528,88.672639,1862.879517,0.177591,2415.721558
24997169,27.541620,88.672824,27.541528,88.672639,1862.879517,0.177623,2415.959057
25004130,27.541528,88.672639,27.541528,88.672639,1862.879517,0.177594,2415.738006
25004131,27.541528,88.672731,27.541528,88.672639,1862.879517,0.177625,2415.975251


time: 904 ms (started: 2021-08-09 06:19:22 +00:00)


In [43]:
final_merged_df=final_merged_df.sort_index()

time: 5.06 s (started: 2021-08-09 06:19:23 +00:00)


In [44]:
final_merged_df=final_merged_df.drop(columns=['latitude_x','longitude_x','latitude_y','longitude_y','predictions','dem height'])
final_merged_df

,height
0,5626.679600
1,5626.801687
2,5626.923717
3,5624.408905
4,5624.531040
...,...
25004128,2384.258483
25004129,2384.495308
25004130,2415.738006
25004131,2415.975251


time: 57 ms (started: 2021-08-09 06:19:28 +00:00)


In [45]:
finalarray = final_merged_df.to_numpy()
finalarray=finalarray.reshape(height*3 ,width*3)
finalarray

array([[5626.67959978, 5626.80168653, 5626.92371749, ..., 5240.18570212,
        5240.16022543, 5240.13471623],
       [5626.9162228 , 5627.03824877, 5627.1602189 , ..., 5240.03607076,
        5240.01065269, 5239.98520213],
       [5627.15265927, 5627.27462439, 5627.3965336 , ..., 5239.88612589,
        5239.86076649, 5239.83537461],
       ...,
       [5179.47869738, 5179.22898416, 5178.97909361, ..., 2415.23146961,
        2415.46876561, 2415.70628988],
       [5179.33557783, 5179.08591227, 5178.8360694 , ..., 2415.48428747,
        2415.721558  , 2415.95905676],
       [5179.1924408 , 5178.942823  , 5178.69302794, ..., 2415.73800634,
        2415.97525128, 2416.21272441]])

time: 7.06 ms (started: 2021-08-09 06:19:29 +00:00)


In [46]:
dst_filename = 'sikkim_SVR.tiff'
x_pixels = width*3  # number of pixels in x
y_pixels = height*3  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float32)
dataset.GetRasterBand(1).WriteArray(finalarray)

0

time: 89.4 ms (started: 2021-08-09 06:19:29 +00:00)


In [47]:
gtlist = list(gt)
gtlist[1]=gtlist[1]/3
gtlist[5]=gtlist[5]/3
gt=tuple(gtlist)

time: 2.62 ms (started: 2021-08-09 06:19:29 +00:00)


In [48]:
geotrans = gt  #get GeoTranform from existed 'data0'
proj=ds.GetProjection() #you can get from a exsited tif or import 
dataset.SetGeoTransform(geotrans)
dataset.SetProjection(proj)
dataset.FlushCache()
dataset=None

time: 138 ms (started: 2021-08-09 06:19:29 +00:00)


In [49]:
from google.colab import files
files.download('sikkim_SVR.tiff')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

time: 16.3 ms (started: 2021-08-09 06:19:29 +00:00)
